### Higher kind type

* `List[Int]`가 아니라 `List`에 대한 Interface를 구축하려면?
  * Not `Iter[List[Int]]`, but `Iter[List]`

In [2]:
// Int: *
// List[_]: * -> *
// List[Int]: *

abstract class Iter[I[_]] {
    def getValue[A](a: I[A]): Option[A]
    def getNext[A](a: I[A]): I[A]
}

abstract class Iterable[R[_]] {
    type Itr[_]
    def iter[A](a: R[A]): Itr[A]
    def iterIF: Iter[Itr]
}

defined class Iter
defined class Iterable

#### *Q: What is the sort (type) of Iter?*
* `3`: `Int`
* `Int`: `*`
* `List`: `*->*`
* `Iter`: `(*->*)->*_`

In [3]:
def sumElements[I[_]](xs: I[Int])(implicit IT: Iter[I]): Int = {
    IT.getValue(xs) match {
        case None => 0
        case Some(n) => n + sumElements(IT.getNext(xs))
    }
}

def printElements[I[_],A](xs: I[A])(implicit IT: Iter[I]): Any = {
    IT.getValue(xs) match {
        case None => 0
        case Some(a) => {
            println(a)
            printElements(IT.getNext(xs))
        }
    }
}

defined function sumElements
defined function printElements

In [4]:
def sumElements2[R[_]](xs: R[Int])(implicit ITR: Iterable[R]) = {
    sumElements(ITR.iter(xs))(ITR.iterIF)
}
def printElements2[R[_],A](xs: R[A])(implicit ITR: Iterable[R]) = {
    printElements(ITR.iter(xs))(ITR.iterIF)
}

defined function sumElements2
defined function printElements2

In [29]:
trait ListIF[L[_]] {
    def empty[A]: L[A]
    def head[A](l: L[A]): Option[A]
    def tail[A](l: L[A]): L[A]
    def cons[A](a: A, l: L[A]): L[A]
    def append[A](l1: L[A], l2: L[A]): L[A]
}
trait TreeIF[T[_]] {
    def empty[A]: T[A]
    def node[A](a: A, l: T[A], r: T[A]): T[A]
    def head[A](t: T[A]): Option[A]
    def left[A](t: T[A]): T[A]
    def right[A](r: T[A]): T[A]
}

defined trait ListIF
defined trait TreeIF

In [12]:
def testList[L[_]](implicit LI: ListIF[L], IT: Iter[L]) {
    val l = LI.cons(3, LI.cons(5, LI.cons(2, LI.cons(1, LI.empty))))
//     println(sumElements[L](l)(IT))
//     printElements[L,Int](l)(IT)
    println(sumElements(l))
    printElements(l)
}
def testTree[T[_]](implicit TI: TreeIF[T], ITR: Iterable[T]) {
    val t = TI.node(3, TI.node(4, TI.empty, TI.empty), TI.node(2, TI.empty, TI.empty))
//     println(sumElements2[T](t)(ITR))
//     printElements2[T,Int](t)(ITR)
    println(sumElements2(t))
    printElements2(t)
}

defined function testList
defined function testTree

In [13]:
implicit def listIter: Iter[List] = {
    new Iter[List] {
        def getValue[A](a: List[A]) = a.headOption
        def getNext[A](a: List[A]) = a.tail
    }
}

implicit def listIF: ListIF[List] = {
    new ListIF[List] {
        def empty[A]: List[A] = Nil
        def head[A](l: List[A]) = l.headOption
        def tail[A](l: List[A]) = l.tail
        def cons[A](a: A, l: List[A]) = a :: l
        def append[A](l1: List[A], l2: List[A]) = l1 ::: l2
    }
}

defined function listIter
defined function listIF

In [14]:
sealed abstract class MyTree[A]
case class Empty[A]() extends MyTree[A]
case class Node[A](value: A, left: MyTree[A], right: MyTree[A]) extends MyTree[A]

implicit def treeIF: TreeIF[MyTree] = {
    new TreeIF[MyTree] {
        def empty[A] = Empty()
        def node[A](a: A, l: MyTree[A], r: MyTree[A]) = Node(a,l,r)
        def head[A](t: MyTree[A]) = t match { 
            case Empty() => None
            case Node(v,_,_) => Some(v)    
        }
        def left[A](t: MyTree[A]) = t match {
            case Empty() => t
            case Node(_,lt,_) => lt    
        }
        def right[A](t: MyTree[A]) = t match {
            case Empty() => t
            case Node(_,_,rt) => rt    
        }  
    }
}

defined class MyTree
defined class Empty
defined class Node
defined function treeIF

In [17]:
def treeIterable[L[_]](implicit IF: ListIF[L], IT: Iter[L]): Iterable[MyTree] = {
    new Iterable[MyTree] {
        // sort Itr = L
        type Itr[X] = L[X]
        def iter[A](a: MyTree[A]): L[A] = a match {
            case Empty() => IF.empty
            case Node(v, left, right) => IF.cons(v, IF.append(iter(left), iter(right)))    
        }
        val iterIF = IT
    }
}

defined function treeIterable

In [24]:
implicit val treeIterableList: Iterable[MyTree] = treeIterable[List]

treeIterableList: Iterable[MyTree] = ammonite.$sess.cmd16$Helper$$anon$1@28f5e6b3

In [25]:
testList[List]

11
3
5
2
1


In [26]:
testTree[MyTree]

9
3
4
2


In [27]:
treeIterableList
implicitly[Iterable[MyTree]]

res26_0: Iterable[MyTree] = ammonite.$sess.cmd16$Helper$$anon$1@28f5e6b3
res26_1: Iterable[MyTree] = ammonite.$sess.cmd16$Helper$$anon$1@28f5e6b3

In [28]:
implicit def iterIterable[I[_]](implicit IT: Iter[I]): Iterable[I] = {
    new Iterable[I] {
        type Itr[A] = I[A]
        def iter[A](a: I[A]): I[A] = a
        def iterIF: Iter[I] = IT
    }
}
val l= List(3,5,2,1)
sumElements2(l)
printElements2(l)

3
5
2
1


defined function iterIterable
l: List[Int] = List(3, 5, 2, 1)
res27_2: Int = 11
res27_3: Any = 0